In [1]:
import pandas as pd
import pandas as pd, numpy as np
from pathlib import Path
import fsspec
import matplotlib.pyplot as plt

Exam Functions

In [ ]:
def station_data(station_id: str,
                 begin_date = "1-Jan-1990",
                 end_date = "31-Dec-2020"):
    
    '''Receives a GHCN Station ID string and returns the daily min
       temperature for the specified period the period 1991-2020'''
    

    # Site hearder plus id path
    download_string = "s3://noaa-ghcn-pds/csv/by_station/" + str(station_id) + ".csv"

    df = pd.read_csv( 
                     download_string,
                     storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
                     dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},
                     parse_dates=['DATE']
                     ).set_index('DATE')
    
    # Filtering for date
    df = df[(df.index >= pd.to_datetime(begin_date)) & (df.index <= pd.to_datetime(end_date))]

    # Selecting only Minimum and Maximum temperature observations
    df_daily_min = df[(df['ELEMENT'] == 'TMIN')]

    result = pd.DataFrame({"daily_min_temp": df_daily_min['DATA_VALUE']/10, 
                           }
                           ) 
    result.index =  df_daily_min.index

    return result



Exam variables

In [21]:
# Station where we are quantifying frost/freeze risk

station = 'USC00087205'

In [ ]:
# Loading station data

df = station_data(station)

C:\Users\12ian\AppData\Local\Temp\ipykernel_26492\1838302253.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [ ]:
# Converting the data to fahrenheit

df['daily_min_temp'] = (df['daily_min_temp']*(9/5)) + 32

In [86]:
# Selecting only data for the months of January, October, November, December

df_J = df[df.index.month == 1]
df_O = df[df.index.month ==10]
df_N = df[df.index.month == 11]
df_D = df[df.index.month == 12]

In [87]:
# Calculating Januray Freeze days
df_J_frz = df_J['daily_min_temp'] <=28
df_J_frz = df_J_frz.groupby(df_J_frz.index.year).sum()

# Calculating January Frost Days
df_J_frst = df_J['daily_min_temp'] <=32
df_J_frst = df_J_frst.groupby(df_J_frst.index.year).sum()

# Calculating October Freeze days
df_O_frz = df_O['daily_min_temp'] <=28
df_O_frz = df_O_frz.groupby(df_O_frz.index.year).sum()

# Calculating October Frost Days
df_O_frst = df_O['daily_min_temp'] <=32
df_O_frst = df_O_frst.groupby(df_O_frst.index.year).sum()

# Calculating November Freeze days
df_N_frz = df_N['daily_min_temp'] <=28
df_N_frz = df_N_frz.groupby(df_N_frz.index.year).sum()

# Calculating November Frost Days
df_N_frst = df_N['daily_min_temp'] <=32
df_N_frst = df_N_frst.groupby(df_N_frst.index.year).sum()

# Calculating December Freeze days
df_D_frz = df_D['daily_min_temp'] <=28
df_D_frz = df_D_frz.groupby(df_D_frz.index.year).sum()

# Calculating December Frost Days
df_D_frst = df_D['daily_min_temp'] <=32
df_D_frst = df_D_frst.groupby(df_D_frst.index.year).sum()

In [88]:
# Calculating the frost risk by taking the mean of each month's frost days

Jan_frost_risk = df_J_frst.mean()
Oct_frost_risk = df_O_frst.mean()
Nov_frost_risk = df_N_frst.mean()
Dec_frost_risk = df_D_frst.mean()

In [89]:
# Calculating the freeze risk by taking the mean of each month's frost days

Jan_freeze_risk = df_J_frz.mean()
Oct_freeze_risk = df_O_frz.mean()
Nov_freeze_risk = df_N_frz.mean()
Dec_freeze_risk = df_D_frz.mean()

In [90]:
print('Frost Risks per Winter Month for Plant City, FL')
print(f'January: {np.round(Jan_frost_risk,2)} days')
print(f'October: {np.round(Oct_frost_risk,2)} days')
print(f'November: {np.round(Nov_frost_risk,2)} days')
print(f'December: {np.round(Dec_frost_risk,2)} days')

Frost Risks per Winter Month for Plant City, FL
January: 1.81 days
October: 0.0 days
November: 0.03 days
December: 0.58 days


In [91]:
print('Freeze Risks per Winter Month for Plant City, FL')
print(f'January: {np.round(Jan_freeze_risk,3)} days')
print(f'October: {np.round(Oct_freeze_risk,3)} days')
print(f'November: {np.round(Nov_freeze_risk,3)} days')
print(f'December: {np.round(Dec_freeze_risk,3)} days')

Freeze Risks per Winter Month for Plant City, FL
January: 0.484 days
October: 0.0 days
November: 0.0 days
December: 0.161 days
